In [22]:
import sys
# sys.path.append("/Users/bubble/Desktop/Project/Infrasound Sensor/Layout/Code/acousticsensor")
import numpy as np
import gdsfactory as gf

cell_temp = gf.Component()

In [23]:
# pad and lines
pad = gf.Component()
pad_size = 500

# lines
line_base = gf.Component()
# define the cross section
w1 = 10
w2 = 2  
s1 = gf.Section(width=w1, offset=0, layer=(2, 0), name="1", port_names=("o1", "o2"))
X1 = gf.CrossSection(sections=[s1])
s2 = gf.Section(width=w2, offset=0, layer=(2, 0), name="1", port_names=("o1", "o2"))
X2 = gf.CrossSection(sections=[s2])
# define the path
# define distance
h1 = 5
P = gf.path.straight(length=h1, npoints=101)

Xtrans = gf.path.transition(X1, X2, width_type='parabolic')
curve_transition = gf.path.extrude_transition(P, Xtrans)

line_base.add_ref(curve_transition).drotate(angle=90, center=(0, 0))
# line_base.show()




Unnamed_166: ports ['o1', 'o2'], vinsts=[] info=Info(length=5.0) kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x115a5b750>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10000'), '5c226817_2000': SymmetricalCrossSection(width=2000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c2268

In [24]:
# pads
pad = gf.Component()
sg_pad = gf.components.rectangle(size=(pad_size, pad_size), layer=(2, 0))
pad.add_ref(sg_pad).move((-pad_size/2, -pad_size))

# pad.show()



Unnamed_168: ports ['e1', 'e2', 'e3', 'e4'], vinsts=[] info=Info() kcl=KCLayout(name='DEFAULT', layout=<klayout.dbcore.Layout object at 0x115a5b750>, layer_enclosures=LayerEnclosureModel(root={'5c226817': LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), '4fb2b992': LayerEnclosure(layer_sections={}, main_layer=9/0, yaml_tag='!Enclosure'), '56396fc8': LayerEnclosure(layer_sections={}, main_layer=SLAB90 (3/0), yaml_tag='!Enclosure'), '6b4bcfc2': LayerEnclosure(layer_sections={}, main_layer=N (20/0), yaml_tag='!Enclosure'), 'a5f0d270': LayerEnclosure(layer_sections={}, main_layer=P (21/0), yaml_tag='!Enclosure')}), cross_sections={'5c226817_10000': SymmetricalCrossSection(width=10000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c226817_10000'), '5c226817_2000': SymmetricalCrossSection(width=2000, enclosure=LayerEnclosure(layer_sections={}, main_layer=SLAB150 (2/0), yaml_tag='!Enclosure'), name='5c22

In [25]:
# cell
L = [250, 500, 750, 1000]
w = 3
block = gf.Component()
block_temp = gf.Component()
origin = [0, 0]

for i in range(len(L)):
    # add beam
    beam = gf.components.rectangle(size=(w, L[i]), layer=(9, 0))
    # length mark
    T = gf.components.text(f"{L[i]}", size=50, layer=(1, 0))
    T_ref = block_temp << T
    # add pad
    vertical_line = gf.components.rectangle(size=(w2, L[i]/2), layer=(2, 0))
    pad_temp = gf.Component()
    pad2 = gf.Component()
    (pad2 << vertical_line).move((-w2/2, h1))
    pad2 << pad
    pad2 << line_base
    pad_temp << pad2
    (pad_temp << pad2).dmirror_y(h1+L[i]/2)
    pad_ref = block_temp << pad_temp
    # add backside frame
    frame_temp = gf.Component()
    size = L[i]
    frame = gf.components.rectangle(size=(size, size), layer=(3, 0))
    frame_temp.add_ref(frame).movex(-size/2+w/2)
    beam_temp = gf.boolean(A = frame_temp, B = beam,  operation="not", layer1=(3, 0), layer2=(9, 0), layer=(1, 0))
    # add backside etch
    size2 = 371.72 + size
    backside_temp = gf.Component()
    backside = gf.components.rectangle(size=(size2, size2), layer=(3, 0))
    backside_temp.add_ref(backside).move((-(size2-size)/2-size/2+w/2, -(size2-size)/2))
    beam_temp << backside_temp
    beam_ref = block_temp << beam_temp
    # adjust position
    beam_ref.movey(2500*i)
    T_ref.movey(2500*i-700)
    pad_ref.move((w/2, 2500*i-h1))

block << block_temp
block.show()

In [26]:
# structure for each die
# structure for a block
fblock = gf.Component()
for i in range(6):
    block_ref = fblock << block
    block_ref.move((2500*(i+1), 0))


In [ ]:
# boolean operation

cell_temp.add_ref(fblock).move((-1000, 1200))
# add order
# cell_temp << order
# frame
frame1 = gf.components.rectangle(size=(15000, 10830), layer=(20, 0))
cell_temp << frame1
frame2 = gf.components.rectangle(size=(20000, 15830), layer=(21, 0))
frame2_ref = cell_temp << frame2
frame2_ref.move((-2500, -2500))

cell_d_clamped_tunable = gf.Component()
cell_ref = cell_d_clamped_tunable << cell_temp
cell_ref.move((2500, -7500))
# cell_d_clamped_tunable.show()
# cell_d_clamped_tunable.write_gds("mesh.gds")
# cell_d_clamped_tunable.plot()